In [14]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
path = DATA_DIR/'sample_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20070101


In [24]:
feed.stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
0,FUR_CREEK_RES,Furnace Creek Resort (Demo),NaN,36.425288,-117.133162,NaN,NaN
1,BEATTY_AIRPORT,Nye County Airport (Demo),NaN,36.868446,-116.784582,NaN,NaN
2,BULLFROG,Bullfrog (Demo),NaN,36.881080,-116.817970,NaN,NaN
3,STAGECOACH,Stagecoach Hotel & Casino (Demo),NaN,36.915682,-116.751677,NaN,NaN
4,NADAV,North Ave / D Ave N (Demo),NaN,36.914893,-116.768210,NaN,NaN
5,NANAA,North Ave / N A Ave (Demo),NaN,36.914944,-116.761472,NaN,NaN
6,DADAN,Doing Ave / D Ave N (Demo),NaN,36.909489,-116.768242,NaN,NaN
7,EMSI,E Main St / S Irving St (Demo),NaN,36.905697,-116.762180,NaN,NaN
8,AMV,Amargosa Valley (Demo),NaN,36.641496,-116.400940,NaN,NaN


In [32]:
f = feed.routes.copy()
cond = f['route_color'].notnull()
f[cond].index.tolist()
#gt.check_routes(F)

[]

In [ ]:
feed.describe(date)

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()